# Simple Code Example

This notebook demonstrates the code to load, parse, prune, and serialize type definitions.

Since ts_type_filter is all about reducing LLM prompt size, we'll
bring in [tiktoken](https://github.com/openai/tiktoken) so that we can measure the number of tokens in
the original and pruned type definitions.

In [1]:
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")

Next we'll implement some simple functions to parse typescript types, index their string literals, and perform pruning.

In [2]:
from ts_type_filter import (
    collect_string_literals,
    build_type_index,
    build_filtered_types,
    parse
)

# Read the contents of menu.ts into a string.
# Use utf8 because of terms like "jalapeño".
filename = "junk.ts"
with open(filename, "r", encoding="utf-8") as file:
  menu_text = file.read()

# Parse the TypeScript type definitions
type_defs = parse(menu_text)

# Create an index of terms in string literals found in the type definitions
symbols, indexer = build_type_index(type_defs)

# Function to prunes the type definitions to include only those branches
#  involvingterms from the shopping cart data structure and user query.
def prune(cart, user_query):
  cart_literals = collect_string_literals(cart)
  full_query = [user_query] + cart_literals
  pruned = build_filtered_types(type_defs, symbols, indexer, full_query)
  return pruned

# Function to print out the token count, followed by the the pruned type
# definitions.
def format(types):
  text = "\n".join([x.format() for x in types])
  encoding = tokenizer.encode(text)
  print(f"Tokens: {len(encoding)}\n")
  print(text)

### Original Menu

First let's look at the unpruned menu.

In [3]:
format(type_defs)

Tokens: 1274

type Cart={items:Item[]};
type Item=WiseguyMeal<ComboSizes>|Meal<ComboSizes>|Wiseguy|PattyMelt|Burger|Chicken|KoreanChicken|Pitas|Fish|ComboTwo|ComboThree|FrenchFries<any>|OtherFries<any,any>|FountainDrink<any,any>;
type WiseguyMeal<SIZE extends ComboSizes>={name:"Wiseguy Meal",size:SIZE,sandwich:Wiseguy|CHOOSE,fries:FrenchFries<SIZE>|CHOOSE,drink:ChooseDrink};
type Meal<SIZE extends ComboSizes>={name:"Meal",size:SIZE,sandwich:Wiseguy|PattyMelt|Burger|Chicken|KoreanChicken|Pitas|Fish|CHOOSE,fries:FrenchFries<SIZE>|CHOOSE,drink:ChooseDrink};
type ComboTwo={name:"Twofer Combo",one:TwoThreeChoices,two:TwoThreeChoices};
type ComboThree={name:"Threefer Combo",one:TwoThreeChoices,two:TwoThreeChoices,three:TwoThreeChoices};
type TwoThreeChoices=Wiseguy|GenericChicken<"Grilled Chicken Sandwich">|GenericBurger<"Bacon Cheeseburger">|FrenchFries<"Medium">|OtherFries<"Jalapeño Poppers","6 Piece">|FountainDrink<"Coca-Cola"|"Diet Coke"|"Dr. Pepper"|"Sprite","Medium">|CHOOSE;
type Wiseg

### Pruning the menu

Now let's prune the menu to retain the portions relevant to the user
query `wiseguy with no tomatoes` when the cart contains a `Coca-Cola`.
Note that the pruned menu has about one sixth the tokens of the unpruned menu.

~~~
TODO: restore `CHOOSE` comment.
~~~

In [4]:
format(prune({"items": [{"name": "Coca-Cola", "size": "Large"}]}, "wiseguy with no tomatoes"))

Tokens: 248

type Cart={items:Item[]};
type Item=WiseguyMeal<ComboSizes>|Wiseguy|FountainDrink<any,any>;
type WiseguyMeal<SIZE extends ComboSizes>={name:"Wiseguy Meal",size:SIZE,sandwich:Wiseguy|CHOOSE,fries:CHOOSE,drink:ChooseDrink};
type ComboSizes="Large"|CHOOSE;
type CHOOSE="CHOOSE";
type Wiseguy=GenericWiseguy<"Wiseguy"|"Vegan Wiseguy"|"Double Wiseguy"|"Triple Wiseguy"|"Down East Wiseguy">;
type GenericWiseguy<NAME>={name:NAME,type:"Regular"|"With Bacon"|"With Cheese"|"With Bacon and Cheese"|CHOOSE,options?:Veggies};
type Veggies={amount:ExtraAmount,name:"Tomato"};
type ExtraAmount="No"|"Regular";
type ChooseDrink=FountainDrink<any,any>|CHOOSE;
type FountainDrink<NAME extends DrinkNames,SIZE extends DrinkSizes>={name:NAME,size:SIZE};
type DrinkNames="Coca-Cola"|"Coca-Cola Zero Sugar";
type DrinkSizes="Large"|CHOOSE;


Note that the alias `coke` will keep prevent `Coca-Cola` and `Coca-Cola Zero Sugar` from getting pruned. 
This is because the `LITERAL` template is used to provide the `coke` alias.

~~~typescript
type DrinkNames =
  | LITERAL<"Coca-Cola", ["coca", "cola", "coke"], false>
  | LITERAL<"Diet Coke", ["coca", "cola"], false>
  | LITERAL<"Coca-Cola Zero Sugar", ["coca", "cola", "coke", "diet"], false>
  | LITERAL<"Dr. Pepper", ["doctor"], false>
  | "Dr. Pepper"
  | "Root Beer"
  | "Diet Root Beer"
  | "Sprite"
  | "Sprite Zero"
  | "Sweetened Tea"
  | "Unsweetened Tea"
  | "Strawberry Lemonade"
  | LITERAL<"Arnold Palmer", ["iced tea lemonade"], false>
  | LITERAL<"Powerade Zero", "Gatoraid", false>;
~~~

Also note that `CHOOSE` is not pruned because the `LITERAL` template is used to pin it.

~~~typescript
// Hint: Use CHOOSE when customer doesn't specify an option
type CHOOSE = LITERAL<"CHOOSE", [], true>;
~~~

In [6]:
format(prune({"items": []}, "a coke"))

Tokens: 66

type Cart={items:Item[]};
type Item=FountainDrink<any,any>;
type FountainDrink<NAME extends DrinkNames,SIZE extends DrinkSizes>={name:NAME,size:SIZE};
type DrinkNames="Coca-Cola"|"Diet Coke"|"Coca-Cola Zero Sugar";
type DrinkSizes="CHOOSE";
